# Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Récupération des articles

In [ ]:
# Charger le fichier CSV
df = pd.read_csv("data/obsinfox.csv")

# Supprimer les doublons d'URL pour ne scraper chaque article qu'une seule fois
unique_urls = df["URL"].drop_duplicates().tolist()

# Dictionnaire pour stocker les articles récupérés
article_texts = {}

# Fonction pour récupérer le texte d'un article
def get_article_text(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}  # Éviter d’être bloqué
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Vérifier les erreurs HTTP
        soup = BeautifulSoup(response.text, "html.parser")

        # Extraire le texte de l'article
        paragraphs = soup.find_all("p")
        article_text = " ".join([p.get_text() for p in paragraphs])
        print(f"✔ Article récupéré : {url}")
        return article_text
    except Exception as e:
        print(f"❌ Erreur avec {url}: {e}")
        return None

# Récupérer les articles uniques
for url in unique_urls:
    article_texts[url] = get_article_text(url)

# Ajouter les textes récupérés à toutes les lignes correspondantes dans le DataFrame
df["Article_Text"] = df["URL"].map(article_texts)

# Sauvegarder les articles récupérés
df.to_csv("data/obsinfox_with_articles.csv", index=False)

print("✅ Extraction terminée !")


✔ Article récupéré : https://lesakerfrancophone.fr/la-relation-entre-la-technologie-et-la-religion
✔ Article récupéré : https://www.breizh-info.com/2021/01/27/157958/confinement-les-habitants-de-brest-morlaix-et-carhaix-invites-a-repondre-a-une-etude-sur-leur-consommation-dalcool/
❌ Erreur avec https://reseauinternational.net/la-chine-le-premier-marche-mondial-de-smartphones/: 403 Client Error: Forbidden for url: https://reseauinternational.net/la-chine-le-premier-marche-mondial-de-smartphones/
✔ Article récupéré : https://lezarceleurs.blogspot.com/2021/12/emmanuel-olivier-tiens-bon-on-les-aura.html
✔ Article récupéré : https://lesakerfrancophone.fr/selon-ubs-les-proprietes-dassurance-tant-de-fois-eprouvees-brillent-plus-que-jamais
❌ Erreur avec https://lesmoutonsrebelles.com/poutine-les-accords-de-minsk-nexistent-plus-et-lukraine-doit-etre-demilitarisee/: 404 Client Error: Not Found for url: https://lesmoutonsrebelles.com/poutine-les-accords-de-minsk-nexistent-plus-et-lukraine-doit-et

In [ ]:
# Trouver les URLs qui ont échoué
failed_urls = [url for url, text in article_texts.items() if text is None]
print(f"❌ {len(failed_urls)} articles non récupérés sur {len(unique_urls)}.")

❌ 8 articles non récupérés sur 100.


# Création du fichier obsinfox_final.csv

### Enlever les lignes avec les textes manquants

In [3]:
df  = pd.read_csv("data/obsinfox_with_articles.csv")
df = df.dropna(subset=['Article_Text'])
df.head()

,URL,Title,Fake News,"Places, Dates, People",Facts,Opinions,Subjective,Reported information,Sources Cited,False Information,Insinuation,Exaggeration,Offbeat Title,Annotator,Article_Text
0,https://lesakerfrancophone.fr/la-relation-entr...,La relation entre la technologie et la religion,0,1,1,1,1,0,1,0,0,0,0,rater1,Par Austin Cline − Le 27 fevrier 2019 − Source...
1,https://www.breizh-info.com/2021/01/27/157958/...,"Confinement. Les habitants de Brest, Morlaix e...",0,1,1,0,0,0,1,0,0,0,0,rater1,"Dans le cadre du Dry January, opération commer..."
2,https://reseauinternational.net/la-chine-le-pr...,La Chine : Le premier marché mondial de Smartp...,0,1,1,0,0,0,1,0,0,0,0,rater1,Le marché des GSM Chinois a évolué en quelques...
3,https://lezarceleurs.blogspot.com/2021/12/emma...,"Emmanuel à Olivier : « Tiens bon, on les aura ...",1,1,1,1,1,0,0,1,0,1,0,rater1,\n De : https://lemediaen442.fr/emmanuel-a-ol...
4,https://lesakerfrancophone.fr/selon-ubs-les-pr...,"Selon UBS, les « propriétés d’assurance tant d...",0,1,1,1,1,0,1,0,0,0,0,rater1,Par Birch Gold Group − Le 17 février 2022 − So...


### Moyenner les valeurs des labels sur tous les annotateurs

In [4]:
df_unique = df.drop_duplicates(subset=['Article_Text'])

In [5]:
# Group by URL and calculate the mean of labels 0 and 1 for all raters
df_fn = df.groupby('URL')['Fake News'].mean().reset_index()
df_facts = df.groupby('URL')['Facts'].mean().reset_index()
df_opinions = df.groupby('URL')['Opinions'].mean().reset_index()
df_subjective = df.groupby('URL')['Subjective'].mean().reset_index()
df_sources = df.groupby('URL')['Sources Cited'].mean().reset_index()
df_reported = df.groupby('URL')['Reported information'].mean().reset_index()
df_false = df.groupby('URL')['False Information'].mean().reset_index()
df_insinuation = df.groupby('URL')['Insinuation'].mean().reset_index()
df_exaggeration = df.groupby('URL')['Exaggeration'].mean().reset_index()
df_offbeat = df.groupby('URL')['Offbeat Title'].mean().reset_index()
df_people = df.groupby('URL')['Places, Dates, People'].mean().reset_index()

#regrouper en un seul df avec le titre, le texte et les scores moyens
df = pd.merge(df_fn, df_facts, on='URL', how='outer')
df = pd.merge(df, df_opinions, on='URL', how='outer')
df = pd.merge(df, df_subjective, on='URL', how='outer')
df = pd.merge(df, df_sources, on='URL', how='outer')
df = pd.merge(df, df_reported, on='URL', how='outer')
df = pd.merge(df, df_false, on='URL', how='outer')
df = pd.merge(df, df_insinuation, on='URL', how='outer')
df = pd.merge(df, df_exaggeration, on='URL', how='outer')
df = pd.merge(df, df_offbeat, on='URL', how='outer')
df = pd.merge(df, df_people, on='URL', how='outer')

df = pd.merge(df, df_unique[['URL', 'Title','Article_Text']], on='URL', how='outer')

### Enregistrement du fichier

In [6]:
# Assuming 'df' is your DataFrame as defined in the provided code.
df.to_csv("data/obsinfox_final.csv", index=False)

In [7]:
df = pd.read_csv("data/obsinfox_final.csv")
df.head()

,URL,Fake News,Facts,Opinions,Subjective,Sources Cited,Reported information,False Information,Insinuation,Exaggeration,Offbeat Title,"Places, Dates, People",Title,Article_Text
0,https://lalettrepatriote.com/deja-marie-en-alg...,0.250,1.0,0.500,0.125,0.875,0.125,0.125,0.625,0.125,0.00,1.000,"Déjà marié en Algérie, il épouse une Française…","Chaque jour, recevez gratuitement nos informat..."
1,https://lalettrepatriote.com/emmanuel-macron-e...,0.125,1.0,0.250,0.000,0.875,0.375,0.000,0.125,0.000,0.00,1.000,Emmanuel Macron et Giorgia Meloni appellent à ...,"Chaque jour, recevez gratuitement nos informat..."
2,https://lalettrepatriote.com/le-pakistan-menac...,0.000,1.0,0.250,0.125,0.875,0.375,0.000,0.125,0.000,0.00,0.750,Le Pakistan menace d’interdire Wikipedia,"Chaque jour, recevez gratuitement nos informat..."
3,https://lalettrepatriote.com/un-couple-loge-un...,0.750,1.0,0.375,0.375,0.500,0.875,0.375,0.625,0.750,0.25,0.625,Un couple loge une famille de migrants : 36.00...,"Chaque jour, recevez gratuitement nos informat..."
4,https://lalettrepatriote.com/ursula-von-der-le...,0.375,1.0,0.375,0.375,0.875,0.625,0.125,0.375,0.500,0.50,1.000,Ursula von der Leyen menace les Italiens à la ...,"Chaque jour, recevez gratuitement nos informat..."
